In [1]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.autograd import Variable
from data import VOCroot
from data import VOC_CLASSES as labelmap
import torch.utils.data as data
from PIL import Image
import sys
import os
from data import AnnotationTransform, VOCDetection, base_transform
from timeit import default_timer as timer
import argparse
import numpy as np
from ssd import build_ssd
import pickle

In [4]:
def test_net(save_folder, net, cuda, valset, transform, top_k, thresh):
    # dump predictions and assoc. ground truth to text file for now
    filename = save_folder+'test1.txt'
    num_images = len(valset)
    recs = {}
    class_recs = {}
    npos = 0

    for i in range(num_images):
        print('Evaluating image {:d}/{:d}....'.format(i+1, num_images))
        img = valset.pull_image(i)
        anno = valset.pull_anno(i)
        print(anno)
        anno = torch.Tensor(anno).long()
        boxes, labels = torch.split(anno, 4, 1) # maybe also return img id
        if(boxes.dim() == 1):
            boxes.unsqueeze_(0)

        det = [False] * boxes.size(0) # detected in given image
        x = Variable(transform(img).unsqueeze_(0))
        if cuda:
            x = x.cuda()
        y = net(x)      # forward pass
        detections = y.data
        # scale each detection back up to the image
        scale = torch.Tensor([img.size[0],img.size[1],img.size[0],img.size[1]])
        pred_num = 0
        # for each class
        for i in range(detections.size(1)):
            j = 0
            # filter out detections that scored less than 0.01
            while detections[0,i,j,0] >= 0.01:
                pt = (detections[0,i,j,1:]*scale)
                pred_num+=1
                
            # compute overlaps
            # if overlap > threshold (0.5)
                # if not found => true positive, mark as found
                # else => duplicate... false positive
            # else => false positive
    # for each class, calc
        # precision (fp / tp+fp)
        # recall (tp+fp / #gt) => gt = tp + fn
    # calc avg prec for each class
    # mAP = mean of APs for all classes


In [5]:
trained_model = 'weights/ssd_300_VOC0712.pth'

# load net
net = build_ssd('test', 300, 21)    # initialize SSD
net.load_state_dict(torch.load(trained_model))
net.eval()
print('Finished loading model!')
# load data
valset = VOCDetection(VOCroot, 'val', None, AnnotationTransform())
# evaluation
test_net('eval/', net, False, valset, base_transform(net.size,(104,117,123)), 5, thresh=0.6)

Finished loading model!
Evaluating image 1/2510....
[[262.0, 210.0, 323.0, 338.0, 8], [164.0, 263.0, 252.0, 371.0, 8], [240.0, 193.0, 294.0, 298.0, 8]]


KeyboardInterrupt: 

In [ ]:
net = build_ssd_v1('test', 300, 21)    # initialize SSD
# net.load_weights('weights/ssd_300_VOC0712.pth')

In [ ]:
x = torch.randn(1,3,300,300)
x = Variable(x)     # wrap tensor in Variable
y = net(x)

In [ ]:
y

In [ ]:
make_dot(y)

In [ ]:
x = torch.Tensor([[1,2,3]])
print(x)
idx = torch.LongTensor([[0,2]])
x.gather(1,idx)[0,:] = 2

In [ ]:
x

In [ ]:
x.scatter_(1,idx,torch.Tensor([[0,0]]))

In [ ]:
x


In [ ]:
idx.size()

In [ ]:
x.size()

# Library Demo

Here we demonstrate framework testing by simulating an SVM classifier and its  
resulting classification accuracy after an attack from the simple_optimize adversary.

## Initial Learner Training
- choose an initial learner model,
- load the instances for training,
- and wrap the two with the Classifier wrapper. 


In [ ]:
# pre-process data and randomly partition
instances = input.load_instances('./data_reader/data/test/100_instance_debug')
random.shuffle(instances)
instances2 = instances[:60]
instances3 = instances[60:]
# print(sparsify(instances)[1].toarray())


# initialize sklearn model
learning_model = svm.SVC(probability=True, kernel='linear')

# initialize and train RobustLearner
clf2 = learner.SimpleLearner(learning_model, instances2)
clf2.train()

# produce simple metrics
y_predict = clf2.predict(instances3)
y_true = sparsify(instances3)[0]
print(y_predict)
print(y_true)
score = metrics.accuracy_score(y_true,y_predict)
print("score = "+str(score))

In [ ]:
# pre-process data and randomly partition
instances = input.load_instances('./data_reader/data/test/100_instance_debug')
random.shuffle(instances)
instances2 = instances[:60]
instances3 = instances[60:]
# print(sparsify(instances)[1].toarray())


# initialize and train RobustLearner
clf2 = learner.SVMFreeRange({'c_f': 0.5}, instances2)
clf2.train()

# produce simple metrics
y_predict = clf2.predict(instances3)
y_true = sparsify(instances3)[0]
print(y_predict)
print(y_true)
score = metrics.accuracy_score(y_true,y_predict)
print("score = "+str(score))

# wgt = clf2.decision_function()[0].tolist()[0]
# print(wgt)
# yaxis = [i for i in range(clf2.num_features)]



In [ ]:
# pre-process data and randomly partition
instances = input.load_instances('./data_reader/data/test/100_instance_debug')
random.shuffle(instances)
instances2 = instances[:60]
instances3 = instances[60:]
# print(sparsify(instances)[1].toarray())

# initialize and train RobustLearner
clf2 = learner.SVMRestrained({'c_delta': 0.5}, instances2)
clf2.train()

# produce simple metrics
y_predict = clf2.predict(instances3)
y_true = sparsify(instances3)[0]
print(y_predict)
print(y_true)
score = metrics.accuracy_score(y_true,y_predict)
print("score = "+str(score))

# wgt = clf2.decision_function()[0].tolist()[0]
# print(wgt)
# yaxis = [i for i in range(clf2.num_features)]

In [ ]:
# plt.figure(figsize=(10,10))
# plt.plot(yaxis, wgt)
# plt.axis([0,1000,-0.00000000005,0.000000000005])
# plt.show()

In [ ]:
clf.train() # train the initial learner

## Adversary Attack 
Now we choose an adversary attack strategy to transform the 
original instances, here we choose simple_optimize.

In [ ]:
# initialize attacker from the ad package
adv = ad.SimpleOptimize(lambda_val=-100, max_change = 65, learner = clf)
# perform the attack on the data to transform instances
instances_post_attack = adv.attack(instances)

### Graph results of pre attack vs. post attack learner 

In [ ]:
# obtain metrics of classification performance both pre- and post- attack
metrics = EvasionMetrics(clf, instances, instances_post_attack)
preAccuracy = metrics.getAccuracy(True)
postAccuracy = metrics.getAccuracy(False)
print ("Pre-attack Accuracy (naive classifier): " + str(preAccuracy))
print ("Post-attack Accuracy: (naive classifier): " + str(postAccuracy))

# plot ROC curve of naive classifier performance after attack
metrics.plotROC(False)

## Learner Response
Here we wrap a defense strategy for the learner to exercise
during the next round of training to become more robust. 

In [ ]:
instances2 = input.load_instances('./data_reader/data/test/100_instance_debug') # load instances
learning_model2 = svm.SVC(probability=True, kernel='linear')
clf2 = Classifier(learning_model2, instances2, ["retraining", adv.clone()]) # could also just pass blank adv instance
clf2.train() # train again with defense strategy 

## Adversary Attack

In [ ]:
# Now we launch the attack on the robust classifier 
adv = ad.SimpleOptimize(lambda_val=-100, max_change = 65, learner = clf2)
instances_post_attack2 = adv.attack(instances2)


### Display metrics after attack on the robust classifier
Here we see the improvements made in accuracy as a result 
of the chosen defense strategy.  Clearly, our learner has 
become more robust to adversary attacks.

In [ ]:
# obtain metrics of classification performance both pre- and post- attack
metrics2 = EvasionMetrics(clf2, instances, instances_post_attack2)
preAccuracy2 = metrics2.getAccuracy(True)
postAccuracy2 = metrics2.getAccuracy(False)
print ("Pre-attack Accuracy (robust classifier): " + str(preAccuracy2))
print ("Post-attack Accuracy: (robust classifier): " + str(postAccuracy2))

metrics2.plotROC(True)